Load expressions
----------------

### imports ###

A hack to use local version o yspecies for debugging

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [1]:
number_of_bootstraps = 5 # this sets global setting of which how many bootstraps to use

lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'max_leaves': 20,
    'max_depth': 3,
    'learning_rate': 0.07,
    'feature_fraction': 0.8,
    'bagging_fraction': 1,
    'min_data_in_leaf': 6,
    'lambda_l1': 0.9,
    'lambda_l2': 0.9,
    "verbose": -1
}

debug_local = True #to use local version

In [2]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
from typing import *
from yspecies.enums import *
from yspecies.dataset import *
from yspecies.misc import *
from yspecies.workflow import *
from yspecies.preprocessing import *
from yspecies.shap_selection import *

In [4]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
import pandas as pd
import shap
from pprint import pprint
import random
import numpy as np
import lightgbm as lgb
from scipy.stats import kendalltau
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, recall_score, precision_score, f1_score

In [6]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

### Loading data ###

In [7]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [8]:
data = ExpressionDataset.from_folder(locations.interim.selected)
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(445, 12243)",12243,39,445,"(67996, 3)","(40, 18)"


In [9]:
from sklearn.pipeline import Pipeline
selection = SelectedFeatures([],[], to_predict = "lifespan")
pipe = Pipeline([
    ('extractor', DataExtractor(selection)), 
    ("partitioner", DataPartitioner()), 
    ("shap_computation", FeatureAnalyzer(ModelFactory()))]
)

In [10]:
d = pipe.fit_transform(data)
d

Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[181]	valid_0's l1: 1.92832	valid_0's l2: 11.964


Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[204]	valid_0's l1: 1.82941	valid_0's l2: 12.1391


Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[228]	valid_0's l1: 2.44614	valid_0's l2: 17.4733


...

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[335]	valid_0's l1: 2.10674	valid_0's l2: 13.204


Setting feature_perturbation = "tree_path_dependent" because no background data was given.


MEAN metrics = R^2    0.977
MSE   14.935
MAE    2.146
dtype: float64


FeatureResults(weighted_features=                ids  gain_score_to_lifespan             name  \
0   ENSG00000242866                 256.330  ENSG00000242866   
1   ENSG00000226979                3654.683  ENSG00000226979   
2   ENSG00000015475                   5.077  ENSG00000015475   
3   ENSG00000060762              106830.922  ENSG00000060762   
4   ENSG00000066923               66911.098  ENSG00000066923   
5   ENSG00000066926                7477.181  ENSG00000066926   
6   ENSG00000073737                   3.359  ENSG00000073737   
7   ENSG00000096080                 217.271  ENSG00000096080   
8   ENSG00000101182                  41.879  ENSG00000101182   
9   ENSG00000101353                 643.524  ENSG00000101353   
10  ENSG00000106066                 953.529  ENSG00000106066   
11  ENSG00000106682                6795.089  ENSG00000106682   
12  ENSG00000117151                6365.475  ENSG00000117151   
13  ENSG00000122034                9038.976  ENSG00000122034   
14  ENSG00000129187                4623.394  ENSG00000129187   
15  ENSG00000131475                  28.160  ENSG00000131475   
16  ENSG00000138615                  30.141  ENSG00000138615   
17  ENSG00000142002                1928.958  ENSG00000142002   
18  ENSG00000152705                3016.753  ENSG00000152705   
19  ENSG00000156467                4154.487  ENSG00000156467   
20  ENSG00000157045                3433.333  ENSG00000157045   
21  ENSG00000163666                2152.743  ENSG00000163666   
22  ENSG00000164045                 729.802  ENSG00000164045   
23  ENSG00000165501                 598.754  ENSG00000165501   
24  ENSG00000165555                 454.861  ENSG00000165555   
25  ENSG00000165689                1851.160  ENSG00000165689   
26  ENSG00000165917                 143.141  ENSG00000165917   
27  ENSG00000167515              121885.874  ENSG00000167515   
28  ENSG00000168060                 276.348  ENSG00000168060   
29  ENSG00000175854              273656.137  ENSG00000175854   
30  ENSG00000176894                7919.762  ENSG00000176894   
31  ENSG00000213139                 942.406  ENSG00000213139   
32  ENSG00000276293              763712.683  ENSG00000276293   
33  ENSG00000083896              231061.620  ENSG00000083896   
34  ENSG00000148248               16510.143  ENSG00000148248   

    kendall_tau_to_lifespan  
0                     0.656  
1                     0.390  
2                     0.244  
3                    -0.239  
4                     0.542  
5                    -0.486  
6                     0.570  
7                    -0.541  
8                    -0.446  
9                     0.746  
10                    0.591  
11                   -0.103  
12                   -0.511  
13                    0.565  
14                    0.656  
15                   -0.806  
16                   -0.253  
17                   -0.449  
18                    0.502  
19                    0.464  
20                   -0.463  
21                    0.590  
22                   -0.685  
23                    0.607  
24                    0.782  
25                   -0.705  
26                   -0.546  
27                   -0.135  
28                    0.571  
29                   -0.651  
30                   -0.390  
31                    0.662  
32                    0.011  
33                    0.066  
34                    0.044  , stable_shap_values=array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), metrics=    R^2    MSE   MAE
0 0.980 11.964 1.928
1 0.979 12.139 1.829
2 0.974 17.473 2.446
3 0.970 19.897 2.419
4 0.980 13.204 2.107)

In [11]:
d.weighted_features

,ids,gain_score_to_lifespan,name,kendall_tau_to_lifespan
0,ENSG00000242866,256.330,ENSG00000242866,0.656
1,ENSG00000226979,3654.683,ENSG00000226979,0.390
2,ENSG00000015475,5.077,ENSG00000015475,0.244
3,ENSG00000060762,106830.922,ENSG00000060762,-0.239
4,ENSG00000066923,66911.098,ENSG00000066923,0.542
5,ENSG00000066926,7477.181,ENSG00000066926,-0.486
6,ENSG00000073737,3.359,ENSG00000073737,0.570
7,ENSG00000096080,217.271,ENSG00000096080,-0.541
8,ENSG00000101182,41.879,ENSG00000101182,-0.446
9,ENSG00000101353,643.524,ENSG00000101353,0.746


In [15]:
show(data.genes_meta, 10)

,Unnamed: 0,species,symbol
gene,,,
ENSG00000242265,0,Homo_sapiens,PEG10
ENSG00000139990,1,Homo_sapiens,DCAF5
ENSG00000073921,2,Homo_sapiens,PICALM


In [18]:
d.stable_shap_values

,R^2,MSE,MAE
0,0.980,11.964,1.928
1,0.979,12.139,1.829
2,0.974,17.473,2.446
3,0.970,19.897,2.419
4,0.980,13.204,2.107


In [2]:
metrics = pd.DataFrame(np.zeros([10,3]), columns=["R^2", "MSE", "MAE"])
metrics

NameError: name 'pd' is not defined

In [3]:
metrics.iloc[4] = np.array([40,40,40])
metrics

NameError: name 'np' is not defined

In [4]:
from yspecies.shap_selection import *
partitioner = DataPartitioner(SelectedFeatures([],[], to_predict = "lifespan"))
factory =  ModelFactory()

ModuleNotFoundError: No module named 'yspecies'

In [12]:
i = 0
for j in range(0, len(partitions.x_partitions)):
    partitions.x_partitions[j] = partitions.x_partitions[j].drop(columns=["species"], errors='ignore')
    
X_test = partitions.x_partitions[i]
y_test = partitions.y_partitions[i]

X_train = pd.concat(partitions.x_partitions[:i] + partitions.x_partitions[i+1:])
y_train = np.concatenate(partitions.y_partitions[:i] + partitions.y_partitions[i+1:], axis=0)

NameError: name 'partitions' is not defined

In [ ]:
show(X_test, 10)

In [ ]:
X_test["ENSG00000139990"].iloc[10:20]

In [ ]:
factory.regression_model(X_train, X_test, y_train, y_test)#, index_of_categorical)

In [ ]:
selected = analyzer.select_features(data)
selected

In [ ]:
def calculate_metrics(prediction, ground_truth):
     return {
            'R2': r2_score(ground_truth, prediction),
            'MSE': mean_squared_error(ground_truth, prediction),
            'MAE': mean_absolute_error(ground_truth, prediction),
     }
    
def encode_tissues(dataframe):
    le.fit(dataframe['tissue'].values)
    tissues_encoded = le.transform(dataframe['tissue'].values)
    dataframe['tissue_encoded'] = tissues_encoded
    
    return dataframe
    
    
def split_to_X_and_Y(dataframe, label_to_predict):
    if 'tissue' in dataframe.columns:
        X = dataframe.drop([label_to_predict, 'tissue'], axis=1)
        Y = dataframe[label_to_predict].values
        index_of_categorical_feature = list(X.columns).index('tissue_encoded')
    else:
        X = dataframe.drop([label_to_predict], axis=1)
        Y = dataframe[label_to_predict].values
        index_of_categorical_feature = None

    return X, X.values, Y, index_of_categorical_feature
    
    
def get_predictions(label_to_predict, ids=None):
    species_data = pd.read_csv('cross_species_df_merged.csv', low_memory=False)
    
    # remove other features (redundant and those that correlate with target)
    cols_to_delete = []
    for column in list(species_data.columns):
        if ids:
            if column not in ids and column not in ['tissue', label_to_predict]:
                cols_to_delete.append(column)
        else:
            if 'ENSG' not in column and column not in ['tissue', label_to_predict]:
                cols_to_delete.append(column)    
            
    species_data = species_data.drop(cols_to_delete, axis=1) 
    
    species_data = species_data[(~pd.isnull(species_data[label_to_predict]))] # select only row where target is set
    species_data = species_data.dropna(axis=1, thresh=int(len(species_data)*0.9)) # remove all genes where percentage of NaN > 10%
    species_data = species_data[species_data['tissue'].isin(['Lung', 'Liver', 'Kidney', 'Brain', 'Heart'])] # remove underrepresented tissues
    species_data = encode_tissues(species_data)
    
    print('Number of samples', len(species_data))
    print('Number of genes', len(species_data.columns))
    
    feature_df, X, Y, index_of_categorical = split_to_X_and_Y(species_data, label_to_predict)
    
    object_from_training = calculate_stable_shap_values(feature_df, Y, index_of_categorical, label_to_predict)
    features_weighted = object_from_training['list_of_weighted_features']
    shap_values = object_from_training['stable_shap_values']
    
    return shap_values, feature_df, features_weighted

### Get list of selected genes for each variable

In [ ]:
lifespan_weighted_features = []
lifespan_shap_values = []
lifespan_dataframes = []

for label in ['gestation_days', 'max_lifespan', 'mass_g', 'temperature_celsius', 'metabolic_rate', 'mtGC']:
    shap_values, feature_df, weighted_features = get_predictions(label)
    lifespan_weighted_features += weighted_features
    lifespan_shap_values.append(shap_values)
    lifespan_dataframes.append(feature_df)
    